# Chapter 15: Processing Sequences Using RNNs and CNNs

Bab ini memperkenalkan **Recurrent Neural Networks (RNNs)**, sebuah kelas jaringan saraf yang dirancang untuk bekerja pada data sekuensial atau data deret waktu (*time series*). Tidak seperti jaringan *feedforward*, RNN memiliki koneksi yang menunjuk ke belakang, memungkinkannya untuk mempertahankan semacam "memori" dari input sebelumnya.

Aplikasi RNN sangat luas, mulai dari memprediksi harga saham, menganalisis data sensor, hingga pemrosesan bahasa alami (NLP) seperti terjemahan mesin atau analisis sentimen.

Kita akan membahas:
* Konsep dasar neuron dan lapisan rekuren.
* Cara melatih RNN menggunakan *Backpropagation Through Time (BPTT)*.
* Kesulitan utama dalam melatih RNN: gradien yang tidak stabil dan memori jangka pendek yang terbatas.
* Arsitektur sel yang lebih canggih seperti **LSTM (Long Short-Term Memory)** dan **GRU (Gated Recurrent Unit)** untuk mengatasi masalah memori.
* Penggunaan **Convolutional Neural Networks (CNNs)** untuk memproses sekuens.

## Recurrent Neurons and Layers

Neuron rekuren (atau sel) tidak hanya menerima input pada waktu `t`, tetapi juga menerima outputnya sendiri dari langkah waktu sebelumnya (`t-1`). Keadaan internal ini, yang disebut *hidden state*, memungkinkan jaringan untuk mengingat informasi dari masa lalu.

Sebuah lapisan RNN terdiri dari beberapa sel rekuren yang bekerja secara paralel, masing-masing mempertahankan *hidden state*-nya sendiri. Lapisan ini dapat memproses sekuens dengan panjang berapa pun.

## Forecasting a Time Series

Salah satu tugas paling umum untuk RNN adalah *forecasting* (peramalan). Kita akan mencoba memprediksi nilai berikutnya dalam sebuah *time series* sintetis.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Fungsi untuk membuat dataset time series
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

# Membuat data
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

### Simple RNN
RNN paling sederhana dapat dibuat menggunakan `SimpleRNN` layer di Keras. Kita akan membangun model untuk memprediksi nilai berikutnya.

In [ ]:
# Membangun model RNN sederhana
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

optimizer = keras.optimizers.Adam(lr=0.005)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

## Handling Long Sequences

RNN sederhana kesulitan belajar dari sekuens yang panjang karena dua alasan utama:
1. **Unstable Gradients:** Seperti DNN biasa, gradien dapat menghilang atau meledak.
2. **Limited Short-Term Memory:** Karena informasi melewati transformasi di setiap langkah, informasi dari langkah-langkah awal dapat hilang.

### LSTM and GRU Cells
Untuk mengatasi masalah memori, sel yang lebih kompleks seperti **LSTM (Long Short-Term Memory)** dan **GRU (Gated Recurrent Unit)** diperkenalkan. Sel-sel ini memiliki mekanisme *gate* (gerbang) yang memungkinkan jaringan untuk belajar apa yang harus diingat, apa yang harus dilupakan, dan apa yang harus dikeluarkan sebagai output. Ini memungkinkan mereka untuk menangkap dependensi jangka panjang dalam data.

In [ ]:
# Membangun model dengan GRU layer
model_gru = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20),
    keras.layers.Dense(1)
])

model_gru.compile(loss="mse", optimizer="adam")
history_gru = model_gru.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

### Using CNNs for Sequences - WaveNet

Arsitektur **WaveNet**, yang awalnya dikembangkan untuk audio, menunjukkan bahwa CNN juga bisa sangat efektif untuk data sekuensial. Ia menggunakan lapisan 1D *convolutional* dengan *dilation rate* yang meningkat secara eksponensial. Ini memungkinkan jaringan untuk memiliki *receptive field* yang sangat besar, memungkinkannya belajar pola jangka panjang dengan cara yang sangat efisien secara komputasi.

### Visualisasi Prediksi

Setelah melatih model, kita dapat memvisualisasikan prediksinya pada data baru untuk melihat seberapa baik kinerjanya.

In [ ]:
# Membuat prediksi pada satu time series dari test set
series_test = X_test[0:1, :, :]
y_pred = model_gru.predict(series_test)

# Fungsi untuk plot
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, "b-")
    if y is not None:
        plt.plot(n_steps, y, "bo", label="Target")
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "rx", markersize=10, label="Prediction")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])
    if y is not None or y_pred is not None:
        plt.legend(fontsize=14)

plot_series(X_test[0, :, 0], y_test[0, 0], y_pred[0, 0])
plt.title("Prediksi Time Series dengan GRU")
plt.show()